In [1]:
%matplotlib widget
from scipy.signal import convolve
from process_cube.dat import read_dat_file, Header
import numpy as np
from tqdm.notebook import tqdm
import numba

def neighbour_mean(x):
    kernel = np.ones((3,3))
    kernel[1,1] =0
    kernel = kernel[...,np.newaxis]
    out = convolve(x, kernel,"same")
    out[0,0]/=3
    out[-1,-1]/=3
    out[-1,0]/=3
    out[0,-1]/=3
    out[0,1:-1]/=5
    out[-1,1:-1]/=5
    out[1:-1,0]/=5
    out[1:-1,-1]/=5
    out[1:-1,1:-1]/=8
    return out

@numba.njit
def estimation(t,phi_inv, data, M):
    est = np.zeros(data.shape[:2])
    for i,row in enumerate(data):
        for j,pixel in enumerate(row):
            est[i,j] = t @ (phi_inv @ (pixel - M[i,j]))
    return est

def covariance_matrix(x):
    covariance = np.zeros((x.shape[-1], x.shape[-1]))
    for i,row in enumerate(x):
        for j,pixel in enumerate(row):
            covariance += pixel.reshape(-1,1) * pixel.reshape(1,-1)
            
    return covariance/x.shape[0]/x.shape[1]

def bins_centers(bins):
    return (bins[:-1] + bins[1:])/2

def get_shared_bins(count, *args):
    low = np.min(args)
    high = np.max(args)
    return np.linspace(low, high, count+1)

def original_algorithm(data, p, t):
    M = neighbour_mean(data)
    data_tag = data + p*t
    phi = covariance_matrix(data-M)
    phi_inv = np.linalg.inv(phi)
    est_no_target = estimation(t,phi_inv, data, M)
    est_with_target = estimation(t,phi_inv, data_tag, M)
    return est_no_target, est_with_target

header = Header.read_header("self_test_rad.hdr")
img = read_dat_file("self_test_rad.img", header).astype(np.float64)
p = 0.02
t = img[60,70,:]
no_target, with_target= original_algorithm(img, p, t)

C:\Users\Shaiel Cohen\AppData\Local\Temp\ipykernel_18808\1841775795.py:29: NumbaPerformanceWarning: '@' is faster on contiguous arrays, called on (Array(float64, 1, 'A', False, aligned=True), Array(float64, 1, 'C', False, aligned=True))
  est[i,j] = t @ (phi_inv @ (pixel - M[i,j]))


In [2]:
import json
from tqdm.notebook import tqdm

@numba.njit
def clustered_estimation(t, data, M, clustered_image, clusters_covariance):
    est = np.zeros(data.shape[:2])
    for i,row in enumerate(data):
        for j,pixel in enumerate(row):
            k = clustered_image[i,j]
            est[i,j] = t @ (clusters_covariance[k] @ (pixel - M[i,j]))
    return est

def clustered_algorithm(data, p, t, clustered_image):
    M = neighbour_mean(data)
    data_tag = data + p*t
    clusters_covariance = np.array([np.linalg.inv(covariance_matrix((data-M)[clustered_image == k][np.newaxis])) for k in tqdm(range(len(cluster_names)))])
    est_no_target = clustered_estimation(t, data, M, clustered_image, clusters_covariance)
    est_with_target = clustered_estimation(t, data_tag, M, clustered_image, clusters_covariance)
    return est_no_target, est_with_target

# Read clusters
with open("clusters.json") as f:
    clusters = json.load(f)

clusters_indexes = {tuple(cluster["name"]): cluster["cluster"] for cluster in clusters.values()}
cluster_names = list(clusters_indexes.keys())
clusters_pixel_index = list(clusters_indexes.values())
clustered_image = np.zeros((img.shape[0], img.shape[1]), dtype=int)

for k, cluster_pixel_index in enumerate(clusters_pixel_index):
    for i,j in cluster_pixel_index:
        clustered_image[i,j] = k

clustered_est_no_target, clustered_est_with_target = clustered_algorithm(img, p, t, clustered_image)

  0%|          | 0/7 [00:00<?, ?it/s]

In [3]:

from matplotlib import pyplot as plt
def get_shared_bins(count, *args):
    low = np.min(args)
    high = np.max(args)
    return np.linspace(low, high, count+1)
bins = get_shared_bins(1000, no_target, with_target)
count_no_target,bins_no_target = np.histogram(no_target.reshape(-1,),bins=bins)
count_with_target,bins_with_target =np.histogram(with_target.reshape(-1,),bins=bins)
fig = plt.figure()
plt.plot(bins_centers(bins_no_target), count_no_target)
plt.plot(bins_centers(bins_with_target), count_with_target)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
from scipy.interpolate import pchip

def get_inv_cdf(count, bins):
    cdf = np.cumsum(count)/np.sum(count)
    return 1 - cdf/cdf[-1]

def extend_cdf_range(samples, sample_points, new_sample_points):
    out = np.zeros(len(new_sample_points))
    current_point_index = 1
    current_point = sample_points[current_point_index]
    current_sample = samples[current_point_index]
    for i, sample_point in enumerate(new_sample_points):
        if sample_point >= current_point:
            current_point_index += 1
        if current_point_index >= len(sample_points):
            current_sample = 0
        else: 
            current_sample = samples[current_point_index-1]
        out[i] = current_sample
    return out

inv_cdf_no_target = get_inv_cdf(count_no_target, bins_no_target)
inv_cdf_with_target = get_inv_cdf(count_with_target, bins_no_target)
value_range = np.sort(np.concatenate((bins_no_target,bins_with_target)))
inv_cdf_no_target = extend_cdf_range(inv_cdf_no_target, bins_no_target, value_range)
inv_cdf_with_target = extend_cdf_range(inv_cdf_with_target, bins_with_target, value_range)
plt.figure()
line, = plt.plot(inv_cdf_no_target, inv_cdf_with_target)
plt.xlim([-0.001,0.1001])
plt.ylim([-0.01,1])
plt.axline((0,0),(0.1,0.1),color=line.get_color(),linestyle="--")
plt.ylabel("P-TP")
plt.xlabel("P-FP")

def lin_interp(x1,y1,x2,y2,x):
    m = (y2-y1)/(x2-x1)
    n = y1
    return m*(x-x1) + n
def A_value(inv_cdf_no_target, inv_cdf_with_target, threshold):
    inv_cdf_no_target = inv_cdf_no_target[::-1]
    inv_cdf_with_target = inv_cdf_with_target[::-1]
    triangle_area = 0.5 * threshold**2
    cutoff = np.argmax(inv_cdf_no_target > threshold)-1
    cutoff = np.max([0,cutoff])
    
    point = lin_interp(
        inv_cdf_no_target[cutoff],
        inv_cdf_with_target[cutoff],
        inv_cdf_no_target[cutoff+1],
        inv_cdf_with_target[cutoff+1],
        threshold
    )
    inv_cdf_no_target = np.concatenate((inv_cdf_no_target[:cutoff], [threshold]))
    inv_cdf_with_target = np.concatenate((inv_cdf_with_target[:cutoff], [point]))
    
    area_under_curve = np.trapz(inv_cdf_with_target, inv_cdf_no_target, )
    
    # area_under_curve = point*threshold/2
    return (area_under_curve - triangle_area)/(threshold - triangle_area)
A_value(inv_cdf_no_target,inv_cdf_with_target,0.1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0.298703661129998

In [5]:
from matplotlib import pyplot as plt
bins = get_shared_bins(1000, clustered_est_no_target, clustered_est_with_target)
count_clustered_no_target,bins_clustered_no_target = np.histogram(clustered_est_no_target.reshape(-1,),bins=bins)
count_clustered_with_target,bins_clustered_with_target =np.histogram(clustered_est_with_target.reshape(-1,),bins=bins)
fig = plt.figure()
plt.plot(bins_centers(bins_clustered_no_target), count_clustered_no_target)
plt.plot(bins_centers(bins_clustered_with_target), count_clustered_with_target)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
from scipy.interpolate import pchip

def get_inv_cdf(count, bins):
    cdf = np.cumsum(count)/np.sum(count)
    return 1 - cdf/cdf[-1]

def extend_cdf_range(samples, sample_points, new_sample_points):
    out = np.zeros(len(new_sample_points))
    current_point_index = 1
    current_point = sample_points[current_point_index]
    current_sample = samples[current_point_index]
    for i, sample_point in enumerate(new_sample_points):
        if sample_point >= current_point:
            current_point_index += 1
        if current_point_index >= len(sample_points):
            current_sample = 0
        else: 
            current_sample = samples[current_point_index-1]
        out[i] = current_sample
    return out

inv_cdf_no_target = get_inv_cdf(count_clustered_no_target, bins_clustered_no_target)
inv_cdf_with_target = get_inv_cdf(count_clustered_with_target, bins_clustered_with_target)
value_range = np.sort(np.concatenate((bins_clustered_no_target,bins_clustered_with_target)))
inv_cdf_no_target = extend_cdf_range(inv_cdf_no_target, bins_clustered_no_target, value_range)
inv_cdf_with_target = extend_cdf_range(inv_cdf_with_target, bins_clustered_with_target, value_range)
plt.figure()
line, = plt.plot(inv_cdf_no_target, inv_cdf_with_target)
plt.xlim([-0.001,0.1001])
plt.ylim([-0.01,1])
plt.axline((0,0),(0.1,0.1),color=line.get_color(),linestyle="--")
A_value(inv_cdf_no_target,inv_cdf_with_target,0.1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0.48192924781503715